# Week 10 Assignment

**Due date**: Tue, Oct 27 by 3:30pm (submit on Sakai)

# Building a bioinformatics pipeline, part I

The goal of this assignment is to create the basis of a bioinformatics pipeline that can be used to study the variation and evolution of genes/proteins.  The pipeline you will build will accomplish the following steps:

1. Read in a CSV delimited spreadsheet containing Genbank accession numbers

2. Use Biopython to download to Genbank files corresponding to the accession numbers in step 1

3. Extract the protein sequences of the genes represented in each genbank file, saving the results into a single FASTA file

4. Align the sequences with the multiple sequence alignment program MUSCLE using the appropriate interface from Biopython

## Data set of interest

We will use our pipeline to study interspecific variation in betacoronavirus spike proteins, and the protein that the spike proteins recognize and bind to on mammalian cells, called ACE2 (Angiotensin-Converting Enzyme 2). For details on background read the following papers:

1. Hu, B., Guo, H., Zhou, P. et al. Characteristics of SARS-CoV-2 and COVID-19. Nat Rev Microbiol (2020). https://doi.org/10.1038/s41579-020-00459-7


2. Broad and Differential Animal Angiotensin-Converting Enzyme 2 Receptor Usage by SARS-CoV-2
Xuesen Zhao, Danying Chen, Robert Szabla, Mei Zheng, Guoli Li, Pengcheng Du, Shuangli Zheng, Xinglin Li, Chuan Song, Rui Li, Ju-Tao Guo, Murray Junop, Hui Zeng, Hanxin Lin
Journal of Virology Aug 2020, 94 (18) e00940-20; https://doi.org/10.1128/JVI.00940-20


# Problems

## 1. Reading list of accessions [5 pts]

Write a function to read a CSV formatted file that contains names and accession numbers for sequences of interest.  Each file will have two columns, "Name" and "Accession", giving the names of the organism that the sequence comes from and the corresponding Genbank accession numbers for each sequence. 

Your function should return a dictionary, with the names (strings) as the keys and the accession numbers (strings) as the values.

The following files give accession list for a set of coronavirus spike proteins and a set of mammalian ACE2 genes:

* `ACE2-accessions.csv`
* `Spike-accessions.csv` (25 Oct 2020: GD-pangolin accession removed)

In [1]:
def read_accessions(csvfile):
    """CSV file with Name and Accession columns -> dictionary relating names to accessions"""
    pass # replace with your code

In [ ]:
# The following tests should work if your read_accessions function works correctly

spikedict = read_accessions("Spike-accessions.csv")
assert(spikedict["RaTG13"] == "MN996532.1")

ace2dict = read_accessions("ACE2-accessions.csv")
assert(ace2dict["Hog badger"] == "MT663962")


## 2. Downloading Genbank files [7.5 pts]

Biopython includes facilities for automatically downloading Genbank files from the web, using NCBI's Entrez interface, as described in [Chap 5](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec59) and [Chap 9](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec%3Aefetch) of the Biopython tutorial.  

**IMPORTANT** Before working on this problem make sure to read the ["Entrez Guidelines"](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec%3Aentrez-guidelines) about how to use the Entrez resources responsibly (if you don't do this you risk having your IP address banned from accessing Entrez). In particular make sure your specify your email address for validating Entrez access after importing the Etnrez library.

``` 
from Bio import Entrez
Entrez.email = "A.N.Other@example.com"  # replace with your Duke email
```

Write a function that takes as input an accession dictionary (e.g. created by the `read_accessions` function). For each accession the function should use Biopython's Entrez tools to download the accessions of interest as genbank files and then each accession out to your file system, with file names formatted as `accession_number.gb`.  Your function should return a dictionary relating the names in the accession dictionary to the filenames of the genbank files.  For examp


In [ ]:
def download_accessions(accessiondict):
    """Given an accession dict, download each accession as a genbank file and return dict
    mapping names to genbank files"""
    pass # replace with your code

In [ ]:
# test your function
testdict = {"Human":"NM_001371415.1"}

testfiledict = download_accessions(testdict)
assert(testfiledict["Human"] == "NM_001371415.1.gb")

# Note that your test should also write a file with the name NM_001371415.1.gb in the 
# directory in your working directory

## Helper function to create Genbank files containing ONLY the spike protein

ADDED: 25 Oct 2020

In the Sunday office hours session, Zach pointed out that the genbank files  for the coronavirus genomes created after completing problem 2, include not only the spike (S) locus but all the genes in the coronavirus genome.  This complicates step 3 below (note that this doesn't apply to the ACE2 accession numbers, because those accessions point directly to the locus of interest).

To ease the burden of completing the subsequent problem, below I provide you with a function to create genbank files that include ONLY the spike proteins. You can then pass the output of this function to your `genbank2fasta` functions below.

In [2]:
def filter_spike_cds(spikeaccessiondict):
    newdict = {}
    for key,value in spikeaccessiondict.items():
        rec = SeqIO.read(value, format="gb")
        spikeftr = None
        for ftr in rec.features:
            if ftr.type == "CDS":
                if "gene" in ftr.qualifiers:
                    if "S" in ftr.qualifiers["gene"]:
                        spikeftr = ftr
                        break
                elif "product" in ftr.qualifiers:
                    for descriptor in ftr.qualifiers["product"]:
                        if "spike" in descriptor:
                            spikeftr = ftr
                            break
        if spikeftr is not None:
            newrec = SeqRecord.SeqRecord(rec.seq, id=rec.id,name=rec.name,
                                         description=rec.description, features=[spikeftr],
                                         annotations=rec.annotations)
            fname = rec.id + "-spikeonly.gb"
            SeqIO.write(newrec, fname, format="gb")
            newdict[key] = fname
    return newdict
    

In [ ]:
spikeonlydict = filter_spike_cds(spikefiledict)  
# use spikeonlydict as an argument to your `genbank2fasta` files below

## 3. Creating FASTA files [7.5 pts]

Your next two function will take the dictionary returned from the `download_accessions` function and then read each file, extracting either the nucleotide sequence or the  translations of the primary coding sequences (CDS) in each file, and will then write out a *single* FASTA file (`fastaname`) with the protein sequences for all the sequences of interest. After parsing the genbank files and writing the corresponding FASTA file, your functions should return the filename of the FASTA file it created.

After calling these functions you should have 2 files for each gene of interest (Spike, ACE2) -- one containing the nucleotide sequences for the gene, the other containing the protein sequences.  Each file should have as many records as there were accessions for that gene.  For example, there are 7 coronavirus accessions so your Spike fasta files will have the spike sequences from those 7 different coronaviruses.  For ACE2, there are 15 accessions (15 different mammal species), so your ACE2 files will have 15 sequences each.


In [ ]:
def genbank2fasta_nucleotide(accessionfiledict, fastaname):
    """Accession file dict -> FASTA file of nucleotide sequences"""
    pass # replace with your code

In [ ]:
def genbank2fasta_protein(accessionfiledict, fastaname):
    """Accession file dict -> FASTA file of amino acid sequences"""
    pass # replace with your code    

## 4. Aligning protein sequences using MUSCLE [5 pts]

[MUSCLE](http://www.drive5.com/muscle/) is a popular multiple sequence alignment program.  You can download a command line version of MUSCLE at that link, and "install" it by unzipping the binary and placing it somewhere in your path (I will show you how to do this in class).

Once MUSCLE is on your computer you should test MUSCLE "by hand" (we'll do this in class).  Biopython also provides an interface to interact with MUSCLE from within a Python pipeline, as described in [Section 6.5](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec92) of the Biopython tutorial.

Using the `MuscleCommandLine` interface that Biopython provides, write a function that aligns the amino acid sequences in a FASTA file:

In [ ]:
def align_via_MUSCLE(infastafile, outfastafile):
    pass # replace with your code
    

## 6. Test the entire pipeline [5 pts]

If each of your functions above works, then the following small code block implements the entire pipeline from start to finish:

In [ ]:
spikedict = read_accessions("Spike-accessions.csv")
spikefiledict = download_accessions(spikedict)

# this only applies to the spike accessions (see note above)
spikeonlydict = filter_spike_cds(spikefiledict)  

spikenucfasta = genbank2fasta_nucleotide(spikeonlydict, "Spike-nucs.fasta")
spikeprotfasta = genbank2fasta_protein(spikeonlydict, "Spike-prot.fasta")
align_via_MUSCLE(spikeprotfasta, "Spike-prot-aligned.fasta")

In [2]:
## Write the corresponding codeblock for the ACE2 sequences